<a href="https://colab.research.google.com/github/fpaterni10/projeto-aplicado-iv-desemprego-br/blob/main/notebooks/entrega2/cd_projeto_aplicado_IV_entrega_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <img src="https://raw.githubusercontent.com/scalabrinig/cdProjetoAplicadoIV/d093146488f56dfcf0ef286bcee8efe0e71b9c76/figuras/mackenzie_logo.jpg" width="25%" align="right"/>

# **PROJETO APLICADO IV - Ciência de Dados EaD - 2025/02**

# **Entrega 2**

---

# **Previsão da Taxa de Desocupação no Estado de São Paulo com Modelos de Séries Temporais**:

---

In [ ]:
#@title **Identificação do Grupo e Opção do Projeto**
#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe: <nome>, <matrícula>*)
Aluno1 = 'Aline Correa de Araújo, 10414773' #@param {type:"string"}
Aluno2 = 'Franciele Paterni, 10414598' #@param {type:"string"}
Aluno3 = 'Giovanna Sobral da Silva, 10424600' #@param {type:"string"}
Aluno4 = 'Guilherme Soares Frota, 10416060' #@param {type:"string"}


# **Introdução**
A taxa de desocupação é um dos principais indicadores socioeconômicos utilizados para medir a saúde do mercado de trabalho. No Brasil, esse índice reflete diretamente o poder de compra, a renda das famílias e a qualidade de vida da população. Segundo o DIEESE e a Fundação SEADE, os movimentos da desocupação estão fortemente associados a fatores macroeconômicos, como política monetária, taxa básica de juros (SELIC), inflação e geração de empregos formais.

Ao longo da última década, a taxa de desemprego tem sofrido oscilações expressivas, principalmente em períodos de crise econômica e de retração do mercado formal. Esses ciclos impactam o consumo interno, aumentam a informalidade e acentuam desigualdades sociais. Por isso, compreender o comportamento da taxa de desocupação e prever sua evolução é estratégico para subsidiar a formulação de políticas públicas, orientar investimentos e apoiar ações de requalificação profissional.

Neste projeto, propomos um modelo preditivo baseado em séries temporais capaz de estimar, mês a mês, a taxa de desemprego no Brasil/Estado de São Paulo, considerando microdados do CAGED (2012–2025) e variáveis exógenas como a taxa SELIC. Essa abordagem combina granularidade dos microdados trabalhistas com fatores macroeconômicos, permitindo análises mais robustas e aplicáveis.

# **Referencial Teórico**

Este tópico sintetiza os conceitos e trabalhos relacionados que sustentam nossa solução (máx. ~1 página).

**Séries temporais** —**Definição e propriedades**: séries temporais são observações ordenadas no tempo; seu tratamento exige atenção à tendência, sazonalidade, autocorrelação e heterocedasticidade. Testes de estacionariedade (ADF, KPSS) e transformações (diferenciação, log) são rotinas essenciais para modelagem adequada.

**Modelos clássicos** — ARIMA/SARIMA/SARIMAX: ARIMA modela componentes autorregressivos (AR), de média móvel (MA) e diferenciação (I). SARIMA estende ARIMA para sazonalidade (com parâmetros sazonais). SARIMAX permite incluir variáveis exógenas (por exemplo, SELIC) para capturar efeitos macroeconômicos sobre a desocupação. Esses modelos têm forte base teórica, interpretabilidade e bom desempenho em séries com estrutura linear e sazonal conhecida.

**Modelos alternativos e ML** — Prophet, XGBoost, RandomForest, LSTM: Prophet (Facebook) facilita modelagem de sazonalidade e feriados com pouca engenharia; modelos baseados em árvores (XGBoost, RandomForest) e redes neurais (LSTM) podem capturar relações não-lineares e interações complexas, porém demandam mais dados/engenharia de features e são menos interpretáveis. Em particular, XGBoost com features de defasagem e estatísticas móveis tem sido usado com sucesso em previsões macroeconômicas recentes.

**Feature engineering em séries** — lags, diferenças e agregações: criação de defasagens (t−1,t−12), médias móveis, variações percentuais e indicadores de sazonalidade (mês do ano) são essenciais para modelos que não trabalham explicitamente com sazonalidade como SARIMA.

**Validação temporal** — Time series cross-validation: usar validação retrógrada (rolling/expanding windows) é crítico para estimar performance realista em horizonte de previsão. Métricas: MAE, RMSE e MAPE são recomendadas para comparar modelos.

**Trabalhos correlacionados**: estudos de previsão do desemprego no Brasil/BR-Estados combinam modelos SARIMA/SARIMAX e modelos de aprendizado de máquina. Exemplos relevantes (resumidos):
- Casagrande et al. (2016) avaliaram métodos ARIMA para taxa de desemprego mensal e discutiram limitações em presença de choques econômicos.  
- Estudos recentes empregam SARIMAX com variáveis macro (SELIC, PIB) para melhorar previsões curtíssimo prazo.  

**Vantagens e limitações**: ARIMA/SARIMA oferecem interpretabilidade e exigem menos dados, mas podem falhar diante de não-linearidades e choques estruturais; modelos ML capturam padrões complexos, mas demandam validação robusta e risco de overfitting. Nossa estratégia híbrida (SARIMAX + ensemble com XGBoost) busca equilibrar interpretabilidade e performance.

Referências citadas no texto seguem padrão ABNT (ver seção Referências).

# **Pipeline da Solução**

Abaixo apresentamos a proposta de pipeline para transformar microdados do CAGED e variáveis exógenas em previsões mensais da taxa de desocupação.

1. **Ingestão e versionamento**
   - Coletar microdados CAGED (2012–2025) e arquivos de SELIC (histórico mensal).  
   - Salvar raw data com versionamento (pasta raw/ ou S3) e registrar data de ingestão.

2. **Validação e pré-processamento**
   - Checagem de integridade (missing, duplicados, consistência de campos).  
   - Padronização de tipos e normalização de códigos (municipio, CNAE, etc.).  
   - Conversão de datas e agregação mensal (saldo: admissões − desligamentos) por recorte desejado (Brasil / SP / RMSP).

3. **Construção da série alvo**
   - A partir do saldo mensal derivar taxa de desocupação (ou usar série oficial como target, se disponível).  
   - Gerar séries de controle: emprego formal, admissões, desligamentos.

4. **Feature engineering**
   - Criar lags: t-1, t-3, t-6, t-12; diferenças (∆1, ∆12).  
   - Estatísticas móveis: média móvel 3/6/12, desvio padrão móvel.  
   - Variáveis exógenas: SELIC (t, t-1), indicador de choques (ex.: choque COVID), dummy de feriados/efeitos sazonais.  
   - One-hot do mês para capturar sazonalidade explícita quando usar modelos ML.

5. **Análise exploratória e testes**
   - Visualização da série (trend + seasonal decomposition).  
   - Testes ADF/KPSS para estacionariedade, ACF/PACF para identificar p/q/P/Q.  
   - Identificação de outliers e tratamento (winsorize ou imputação).

6. **Modelagem**
   - Baseline: média móvel e ARIMA simples.  
   - Modelos principais: SARIMA, SARIMAX (com SELIC), Prophet (opcional).  
   - Modelos alternativos: XGBoost / LightGBM com features de defasagem; LSTM se necessário.  
   - Uso de ensembles simples (média ponderada) para robustez.

7. **Validação e seleção**
   - Validação time-series (rolling windows).  
   - Métricas: MAE, RMSE, MAPE.  
   - Diagnóstico residual: autocorrelação dos resíduos, heterocedasticidade.

8. **Previsão e interpretação**
   - Gerar forecast para horizonte (ex.: 3, 6, 12 meses); calcular intervalos de confiança.  
   - Interpretar impacto das variáveis exógenas (coeficientes SARIMAX; SHAP para XGBoost).

9. **Entrega e documentação**
   - Notebooks limpos com pipeline reproducível, dataset final tratado (.parquet/.csv) e scripts de treinamento.  
   - Relatório em PDF/Markdown com metodologia, resultados e limitações.  
   - Apresentação (slides) e repositório Git versionado.

10. **Automação / Deploy (opcional)**
   - Salvar modelos em disco (pickle/ONNX), criar script para regenerar previsões mensais e dashboard mínimo (Streamlit/Voila) para visualização.

**Observações técnicas**: priorizamos SARIMAX por interpretabilidade e inclusão direta de SELIC; paralelamente treinaremos um XGBoost com features de defasagem para comparar. A seleção final do modelo considera trade-off entre erro preditivo e transparência (relevante para uso público).

# **Cronograma**

A seguir um cronograma sugerido em semanas com prazos (ajuste conforme data de entrega). O ponto de partida adotado aqui é **25/09/2025** (data atual). Substitua as datas conforme necessário.

| Semana | Período | Atividade | Entregável |
|---:|---|---|---|
| Semana 1 | 25/09/2025 – 01/10/2025 | Ingestão, inventário de dados e validação inicial. | Dados brutos versionados + log de ingestão |
| Semana 2 | 02/10/2025 – 08/10/2025 | Tratamento, agregação mensal e construção da série alvo; EDA inicial. | Base mensal consolidada (.parquet) + notebook EDA |
| Semana 3 | 09/10/2025 – 15/10/2025 | Feature engineering (lags, diffs, SELIC) e testes de estacionariedade. | Dataset final para modelagem + relatório curto de testes ADF/KPSS |
| Semana 4 | 16/10/2025 – 22/10/2025 | Treinamento de modelos (SARIMA/SARIMAX, baseline, XGBoost) e validação temporal. | Modelos treinados + métricas (MAE, RMSE, MAPE) |
| Semana 5 | 23/10/2025 – 29/10/2025 | Análise residual, ajuste fino, geração de forecast e interpretações (coeficientes/SHAP). | Previsões para horizonte solicitado + análise interpretativa |
| Semana 6 | 30/10/2025 – 05/11/2025 | Documentação final, preparação de relatório e apresentação (slides). | Relatório (PDF/Markdown) + slides de apresentação |

**Atividades de documento e entrega:** revisão final do notebook, inclusão de referências ABNT, verificação de reproducibilidade e criação de `README.md` com instruções para reproduzir o pipeline.

Se precisar, posso transformar esse cronograma em um gráfico Gantt simples (PNG) ou uma planilha Excel/CSV para controlar tarefas.

# **Referências**

- OLIVEIRA, R.; ALBARRACIN, O. Y.; SILVA, G. R. *Introdução às Séries Temporais: Uma Abordagem Prática em Python*. 2023. Disponível em: <https://github.com/Introducao-Series-Temporais-em-Python/Book>. Acesso em: set. 2025.
- CASAGRANDE, D. L.; OLIVEIRA, F. R.; STUDART, G.; SILVA, I. B.; GUIMARÃES, P. H. M. *Métodos de previsão para a taxa de desemprego mensal: uma análise de séries temporais*. Revista de Economia, Anápolis-GO, v. 12, n. 1, p. 58-86, jan./jun. 2016.
- FUNDAÇÃO SEADE; DIEESE. *Pesquisa de Emprego e Desemprego – Região Metropolitana de São Paulo*. São Paulo, 2012–2018. Disponível em: <https://www.seade.gov.br/>. Acesso em: set. 2025.
- BANCO CENTRAL DO BRASIL. *Taxa SELIC – histórico mensal*. Brasília, 2012–2025. Disponível em: <https://www.bcb.gov.br/estatisticas/txjuros>. Acesso em: set. 2025.
- TIBSHIRANI, R. et al. *An introduction to statistical learning*. Springer, 2ª ed., 2022. (capítulos sobre séries temporais e métodos de ML aplicados a séries)

As referências devem ser formatadas na norma ABNT final no relatório; acima estão em formato reduzido para facilitar edição.

In [ ]:
#@title **Avaliação**
Referencial_teorico = 10 #@param {type:"slider", min:0, max:10, step:1}

Pipeline_solucao = 10 #@param {type:"slider", min:0, max:10, step:1}

Cronograma = 10 #@param {type:"slider", min:0, max:10, step:1}

In [ ]:
#@title **Nota Final**
nota = 0.45*Referencial_teorico + 0.45*Pipeline_solucao + 0.10*Cronograma

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_nome = []

for i in range(1,6):
  try:
    exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_nome.append(lista[0]);")
  except Exception:
    pass

alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)